In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
path = r'C:\Users\muham\OneDrive\Desktop\careerfoundry\Data Immersion\Achievement 6\05-10-25 Accident_data_germany_2024_analysis'

In [5]:
df_clean = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_cleaned.csv'), low_memory= False)

## Mapping state names

In [6]:
state_mapping = {
  1: "Schleswig-Holstein",
2:  "Hamburg",
3:  "Niedersachsen",
4:  "Bremen",
5:  "Nordrhein-Westfalen",
6:  "Hessen",
7:  "Rheinland-Pfalz", 
8:  "Baden-Württemberg",
9:  "Bayern",
10:  "Saarland",
11:  "Berlin", 
12:  "Brandenburg",
13:  "Mecklenburg-Vorpommern", 
14:  "Sachsen",
15:  "Sachsen-Anhalt",
16:  "Thüringen"
}


In [7]:
# creating state name column for clarity in visualizations
df_clean['state_name'] = df_clean['state'].map(state_mapping)



In [8]:
df_clean[['state', 'state_name']].drop_duplicates()


,state,state_name
0,1,Schleswig-Holstein
2675,2,Hamburg
7677,3,Niedersachsen
22375,5,Nordrhein-Westfalen
22694,4,Bremen
60608,6,Hessen
72160,7,Rheinland-Pfalz
78276,8,Baden-Württemberg
98000,9,Bayern
121081,11,Berlin


## Adding state population column

In [9]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268519 entries, 0 to 268518
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     268519 non-null  int64  
 1   OID_                           268519 non-null  int64  
 2   accident_id                    268519 non-null  object 
 3   state                          268519 non-null  int64  
 4   administrative_region          268519 non-null  int64  
 5   administrative_district        268519 non-null  int64  
 6   municipality                   268519 non-null  int64  
 7   year_of_accident               268519 non-null  int64  
 8   month_of_accident              268519 non-null  int64  
 9   hour_of_accident               268519 non-null  int64  
 10  day_of_week                    268519 non-null  int64  
 11  accident_category              268519 non-null  int64  
 12  collision_type                

In [10]:
# population by state
population = {
    'Baden-Württemberg': 11300000,
    'Bayern': 13400000,
    'Berlin': 3850000,
    'Brandenburg': 2540000,
    'Bremen': 690000,
    'Hamburg': 1900000,
    'Hessen': 6400000,
    'Mecklenburg-Vorpommern': 1610000,
    'Niedersachsen': 8030000,
    'Nordrhein-Westfalen': 17900000,
    'Rheinland-Pfalz': 4100000,
    'Saarland': 980000,
    'Sachsen': 4040000,
    'Sachsen-Anhalt': 2170000,
    'Schleswig-Holstein': 2930000,
    'Thüringen': 2090000
}


# Convert dictionary to DataFrame
pop_df = pd.DataFrame(list(population.items()), columns=['state_name', 'population'])


### Merge population column to original data frame

In [11]:
# Merge population with original df_cleaned
df_clean = df_clean.merge(pop_df, on='state_name', how='left')

# Check the first few rows
df_clean[['state_name', 'population']].head()

,state_name,population
0,Schleswig-Holstein,2930000
1,Schleswig-Holstein,2930000
2,Schleswig-Holstein,2930000
3,Schleswig-Holstein,2930000
4,Schleswig-Holstein,2930000


### calculating accidents per 100k population

In [12]:
# Count accidents per state
df_clean['accidents_in_state'] = df_clean.groupby('state')['accident_id'].transform('count')

# Calculate accidents per 100k population
df_clean['accidents_per_100k'] = (df_clean['accidents_in_state'] / df_clean['population']) * 100000

# Inspect
df_clean[['state_name', 'accidents_in_state', 'population', 'accidents_per_100k']].drop_duplicates()


,state_name,accidents_in_state,population,accidents_per_100k
0,Schleswig-Holstein,11731,2930000,400.375427
2675,Hamburg,7031,1900000,370.052632
7677,Niedersachsen,26451,8030000,329.402242
22375,Nordrhein-Westfalen,59035,17900000,329.804469
22694,Bremen,2609,690000,378.115942
60608,Hessen,17947,6400000,280.421875
72160,Rheinland-Pfalz,12164,4100000,296.682927
78276,Baden-Württemberg,31405,11300000,277.920354
98000,Bayern,46446,13400000,346.611940
121081,Berlin,12428,3850000,322.805195


In [13]:
# exporting data frame
df_clean.to_csv(os.path.join(path, '02 Data','Prepared Data', 'df_merged.csv'))